In [ ]:
#| default_exp  Features.distance_angle

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Distance and Angle Feature

In this module, we compute the distance and the angle feature as described in <cite id="g6ews"><a href="#zotero%7C7448072%2FYJQRGD6I">(Fernández, Bornn, and Cervone 2021)</a></cite>. 

First we will compute the `distance` related features :
* Distance between player and ball:

$d_{(p,b)} = \sqrt{(x_{p} - x_{b})^2 + (y_{p} - y_{b})^2}$ with $p$ = player and $b$ = ball.

* Distance between player and its opponent goal:

$d_{(p,g)} = \sqrt{(x_{p} - x_{g})^2 + (y_{p} - y_{g})^2}$ with $p$ = player and $g$ = goal.

* Distance between the ball and the goal:

$d_{(b,g)} = \sqrt{(x_{b} - x_{g})^2 + (y_{b} - y_{g})^2}$ with $b$ = ball and $g$ = goal.

* Distance between the ball and the goalkeeper in y-axis:

$y_{distance(gk,b)} = |y_{gk} - y_{b}|$ with $gk$ = goalkeeper and $b$ = ball.

Second, we will compute the `angle` related features :

* Angle between the ball and the goal :

$deg_{(ball,goal)} = \cos^{-1}(cos(\theta))$

*  Sine of the angle between every location and the ball location and Sine of the angle between the ball carrier velocity vector and every other location:

$sin(\theta) = \sqrt{1 - cos^2(\theta)}$

*  Cosine of the angle between every location and the ball location and Cosine of the angle between the ball carrier velocity vector and every other location:

$cos(\theta) = \frac{\vec{v}_{l,g_{1}}} {\vec{v}_{l,g_{2}}}$ with $l$ = location point to compute angle, $g_{1}$ = higher point of soccer goal and $g_{2}$ = lower point of soccer goal.

`Note:` In our case the pitch dimension field is $(105,68)$ in meters, the attacking team middle goal location is $(0,34)$ where $(0,30.34)$ and $(0,37.66)$ are respecfully the lower and higher points of the goal. The opponent team middle goal location is $(105,34)$ where $(105,30.34)$ and $(105,37.66)$ are respecfully the lower and higher points of the goal.

In [ ]:
#| export

import collections
import math
import os
import re

import numpy as np
import pandas as pd
from fastcore.foundation import L

As usual, we start by reading some data from local disk. We will pick a particular event from the `tracking/event` mapping and select the associated frame:

In [ ]:
#|eval: false

tracking_event_path = "../data/tracking_event_mapping.csv"
tracking_path = "../data/tracking_vel_df.csv"
lineup_path = "../data/lineup.csv"

tracking_event_mapping = pd.read_csv(tracking_event_path, low_memory=False)
tracking_df = pd.read_csv(tracking_path, low_memory=False).set_index("frameId")

## pick the frame-id associated with a randomly picked event
frame_id = tracking_event_mapping.sample(n=1).frameId.values[0]

## select the frame
frame = tracking_df.loc[frame_id]
lineup = pd.read_csv(lineup_path)

## `Distance` and `Angle` class

In [ ]:
#| export


class Dist_ang:
    def __init__(
        self,
        frame: pd.DataFrame(), # one frame DataFrame
        events_to_frame: pd.DataFrame(), # event mapping DataFrame
        lineup: pd.DataFrame(), # lineup DataFrame
    ):
        """ Distance and Angle class to compute features"""
        self.frame, self.lineup, self.events_to_frame = frame, lineup, events_to_frame
        self.gk_ids = lineup[lineup.position == "GK"].playerId.tolist()
        self.frame_id = frame.name
        self.possession_team_id = events_to_frame.loc[
            events_to_frame.frameId == self.frame_id, "teamId"
        ].values[0]
        self.possession_side = self.lineup.loc[
            self.lineup.teamId == self.possession_team_id, "side"
        ].values[0]
        self.opponent_side = "away" if bool(re.match("home",self.possession_side)) else "home"
        self.ball_carrier_id = events_to_frame.loc[
            events_to_frame.frameId == self.frame_id, "playerId"
        ].values[0]
        self.output = {"frameId": self.frame_id}
        
    def _prepare_inputs(self):
        "Prepare necessary inputs"
        _player_id_cols = self.frame.filter(like="_player_id").index.tolist()
        linputs = L(
                {
                    "playerId": self.frame[pid_col],
                    "playerStr": pid_col.removesuffix("_player_id"),
                    "jerseyNumber": int(
                        pid_col.removesuffix("_player_id")
                        .removeprefix("away_player_")
                        .removeprefix("home_player_")
                    ),
                    "side": "home" if bool(re.match("home", pid_col)) else "away",
                    "gy_post1": 30.34,
                    "gy_post2": 37.66,
                    "gx": 105 if bool(re.match("home", pid_col)) else 0,
                    "gy": 34,
                    "x": self.frame[pid_col.removesuffix("_player_id") + "_x"],
                    "y": self.frame[pid_col.removesuffix("_player_id") + "_y"],
                    "vx": self.frame[pid_col.removesuffix("_player_id") + "_vx"],
                    "vy": self.frame[pid_col.removesuffix("_player_id") + "_vy"],
                }
                for pid_col in _player_id_cols
            )
        
        self.inputs = pd.DataFrame(linputs).dropna(axis=0)
        
        self.home_gk_id = self.gk_ids[0] if self.inputs[self.inputs.playerId == self.gk_ids[0]].side.values[0] == "home" else self.gk_ids[1]
        self.away_gk_id = self.gk_ids[0] if self.inputs[self.inputs.playerId == self.gk_ids[0]].side.values[0] == "away" else self.gk_ids[1]
        
    def dist(self,point1,point2):
        "compute distance between two points"
        return math.sqrt( 
            (point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 
        )
    
    def angle(self,point1,point2,angle_point):
        "compute angle, cos and sin between three points"
        a = np.array(point1)
        b = np.array(angle_point)
        c = np.array(point2)
    
        ba = a - b
        bc = c - b
    
        cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cos_angle)
        sin_angle = np.sin(angle)
        
        return np.degrees(angle), cos_angle, sin_angle
    
    def _distance(self):
        "compute distance between every player and ball"
        pl_to_ball = {
            col+"_ball_dist": self.dist(
                [x,y],
                [
                    self.frame["ball_x"],
                    self.frame["ball_y"]
                ]
            )
            for col,x,y in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
            ) 
        }
        
        "compute distance between every player and goal"
        pl_to_goal = {
            col+"_goal_dist": self.dist(
                [x,y],
                [gx,gy]
            )
            for col,x,y,gx,gy in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
                self.inputs.gx.tolist(),
                self.inputs.gy.tolist(),
            ) 
        }
        

        self.output.update(pl_to_ball)
        self.output.update(pl_to_goal)
        
        "compute distance between the ball and the goalkeeper in y-axis"
        self.output[self.inputs[self.inputs.playerId == self.home_gk_id].playerStr.values[0] +
                "_ball_y_dist"] = self.frame["ball_y"] - self.inputs[
            self.inputs.playerId == self.home_gk_id].y.values[0]
        
        self.output[self.inputs[self.inputs.playerId == self.away_gk_id].playerStr.values[0] +
                "_ball_y_dist"] = self.frame["ball_y"] - self.inputs[
            self.inputs.playerId == self.away_gk_id].y.values[0]  
        
        "compute distance between ball and goal"
        self.output["ball_homegoal_dist"] = self.dist(
            [
                self.inputs[self.inputs.playerId == self.home_gk_id].x.values[0],
                self.inputs[self.inputs.playerId == self.home_gk_id].y.values[0]
            ],
            [0,34]
        )
        self.output["ball_awaygoal_dist"] = self.dist(
            [
                self.inputs[self.inputs.playerId == self.away_gk_id].x.values[0],
                self.inputs[self.inputs.playerId == self.away_gk_id].y.values[0]
            ],
            [105,34]
        )
        
    def _angle(self):
        "compute all angle features"
        
        self.output["ball_homegoal_angle"] = self.angle(
            [0,30.34],
            [0,37.66],
            [
                self.frame["ball_x"],
                self.frame["ball_y"]
            ],
        )[0]
        
        self.output["ball_awaygoal_angle"] = self.angle(
            [105,30.34],
            [105,37.66],
            [
                self.frame["ball_x"],
                self.frame["ball_y"]
            ],
        )[0]
        
        pl_goal_angle = {
            col+"_goal_angle": self.angle(
                [gx,gy_post1],
                [gx,gy_post2],
                [x,y]
            )[0]
            for col,x,y,gx,gy_post1,gy_post2 in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
                self.inputs.gx.tolist(),
                self.inputs.gy_post1.tolist(),
                self.inputs.gy_post2.tolist(),
                
            )
        }
        
        pl_goal_cos = {
            col+"_goal_cos": self.angle(
                [gx,gy_post1],
                [gx,gy_post2],
                [x,y]
            )[1]
            for col,x,y,gx,gy_post1,gy_post2 in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
                self.inputs.gx.tolist(),
                self.inputs.gy_post1.tolist(),
                self.inputs.gy_post2.tolist(),
            ) 
        }
        
        pl_goal_sin = {
            col+"_goal_sin": self.angle(
                [gx,gy_post1],
                [gx,gy_post2],
                [x,y]
            )[2]
            for col,x,y,gx,gy_post1,gy_post2 in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
                self.inputs.gx.tolist(),
                self.inputs.gy_post1.tolist(),
                self.inputs.gy_post2.tolist(),
            )
        }

        pl_vballcarrier_cos = {
            col+"_vballcarrier_cos": self.angle(
                [x,y],
                
                [
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].vx.values[0],
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].vy.values[0]
                ],
                
                [
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].x.values[0],
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].y.values[0]
                ]
            )[1]
            for col,x,y in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
            )
        }
        
        pl_vballcarrier_sin = {
            col+"_vballcarrier_sin": self.angle(
                [x,y],
                
                [
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].vx.values[0],
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].vy.values[0]
                ],
                
                [
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].x.values[0],
                    self.inputs[self.inputs.playerId == self.ball_carrier_id].y.values[0]
                ]
            )[2]
            for col,x,y in zip(
                self.inputs.playerStr.tolist(),
                self.inputs.x.tolist(),
                self.inputs.y.tolist(),
            )
        }
        
        self.output.update(pl_goal_angle)
        self.output.update(pl_goal_cos)
        self.output.update(pl_goal_sin)
        self.output.update(pl_vballcarrier_cos)
        self.output.update(pl_vballcarrier_sin)
        
        self.df = pd.DataFrame(self.output,index=[0])

In [ ]:
#|eval: false

dist = Dist_ang(frame, tracking_event_mapping, lineup)
dist._prepare_inputs()
dist.inputs

,playerId,playerStr,jerseyNumber,side,gy_post1,gy_post2,gx,gy,x,y,vx,vy
1,959d1756675b93dfda464e3c7c5edf58f038f0608006b4...,away_player_12,12,away,30.34,37.66,0,34,51.07,28.126377,0.728571,-0.168944
2,957e96878335293e1b2d97a906ccf182c725390214158f...,away_player_15,15,away,30.34,37.66,0,34,74.36,19.049855,-0.514286,1.942857
3,6789255fa015dd9f081e05d224552ca72e61afd17309ed...,away_player_19,19,away,30.34,37.66,0,34,56.42,20.528116,-1.628571,1.224845
4,d8db35953e75ba6c25c946c1fadea2dca90114fccbe767...,away_player_1,1,away,30.34,37.66,0,34,91.13,31.772754,-0.128571,0.549068
6,fb80b18384aa51fa727536f3f44101528774e988c3aee3...,away_player_24,24,away,30.34,37.66,0,34,71.52,39.489275,-0.171429,1.267081
8,d852020e305114abbdef2c6680626fd9743629a930f194...,away_player_29,29,away,30.34,37.66,0,34,56.15,44.426667,2.100000,-0.886957
9,4fdd6d44f27fccd6926b99b66b96ac31acb58d78aa3c5a...,away_player_3,3,away,30.34,37.66,0,34,68.88,10.475942,0.728571,1.478261
10,27a873ab8064d003025d1809775a404ea6d66f3ce24ea8...,away_player_4,4,away,30.34,37.66,0,34,72.73,29.387826,-0.814286,0.802484
11,a093f6b55254780919cddeec4b607b4b72b97497f7acfa...,away_player_5,5,away,30.34,37.66,0,34,63.01,19.976232,0.685714,2.027329
12,cedfe37314f4cda689006461498a507b1b33009e076f5c...,away_player_7,7,away,30.34,37.66,0,34,62.34,30.491594,-2.357143,1.267081


In [ ]:
#|eval: false

dist._distance()
dist._angle()
dist.output

/tmp/ipykernel_234666/84978132.py:66: RuntimeWarning: invalid value encountered in double_scalars
  cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))


{'frameId': 17876,
 'away_player_12_ball_dist': 7.668250220009316,
 'away_player_15_ball_dist': 20.937815880302058,
 'away_player_19_ball_dist': 2.9380548585087407,
 'away_player_1_ball_dist': 39.18227455708127,
 'away_player_24_ball_dist': 25.923324156205517,
 'away_player_29_ball_dist': 23.721815791470192,
 'away_player_3_ball_dist': 18.55356166032694,
 'away_player_4_ball_dist': 21.038784816658183,
 'away_player_5_ball_dist': 9.550361613694061,
 'away_player_7_ball_dist': 13.078290124851648,
 'home_player_11_ball_dist': 19.82649619513648,
 'home_player_17_ball_dist': 14.149729598867504,
 'home_player_1_ball_dist': 37.61535112789312,
 'home_player_20_ball_dist': 35.122839129894146,
 'home_player_2_ball_dist': 13.931190448694265,
 'home_player_3_ball_dist': 12.022296426355448,
 'home_player_4_ball_dist': 26.300159091571405,
 'home_player_5_ball_dist': 16.085632989685205,
 'home_player_8_ball_dist': 0.991164029394377,
 'home_player_9_ball_dist': 14.07010088447808,
 'away_player_12_goal

⚠️ the `nan` value is due to computing the cos & sin angle of the player who carry the ball, its velocity vector and the player itself.

In [ ]:
#|eval: false

dist.df

,frameId,away_player_12_ball_dist,away_player_15_ball_dist,away_player_19_ball_dist,away_player_1_ball_dist,away_player_24_ball_dist,away_player_29_ball_dist,away_player_3_ball_dist,away_player_4_ball_dist,away_player_5_ball_dist,...,home_player_11_vballcarrier_sin,home_player_17_vballcarrier_sin,home_player_1_vballcarrier_sin,home_player_20_vballcarrier_sin,home_player_2_vballcarrier_sin,home_player_3_vballcarrier_sin,home_player_4_vballcarrier_sin,home_player_5_vballcarrier_sin,home_player_8_vballcarrier_sin,home_player_9_vballcarrier_sin
0,17876,7.66825,20.937816,2.938055,39.182275,25.923324,23.721816,18.553562,21.038785,9.550362,...,0.013525,0.91025,0.562891,0.908771,0.975989,0.058291,0.990564,0.773327,NaN,0.168477


In [ ]:
# | hide
from nbdev import nbdev_export
nbdev_export()